# **INTRODUCTIONS**

1. **[Stored Procedures](#stored-procedures)**


In [ ]:
# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/utils/conversions.ipynb",
        "src/utils/schemas.ipynb",
        "src/utils/protos.ipynb"
    ]
}
```

# **COR-1173**

---
This source addresses isue COR-1173, concerning a change in agegroups in a RIVM-file. In the new format the agegroup 0-14 is detailed into the agegroups 0-4, 5-9 and 10-14. In our output, we only distinguish 0-19, so all we need to do here is add a few lines in the Stored Procedure that fills the table in VWSDEST so it recodes all three new agegroups to 0-19. 

The source code contains a single section:


In [ ]:
-- Copyright (c) 2021 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport. 
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.

CREATE OR ALTER   PROCEDURE [dbo].[SP_RIVM_HOSPITAL_IC_ADMISSIONS_OVERTIME_BYAGEGROUP_DEST]
AS
BEGIN			
WITH BASE_CTE AS (
    SELECT 
	   [DATE_OF_REPORT]
      ,[DATE_OF_STATISTICS_WEEK_START]
      ,CASE 
      	WHEN [AGE_GROUP] = '0-4'  THEN '0-19'
	    WHEN [AGE_GROUP] = '5-9'  THEN '0-19'
	    WHEN [AGE_GROUP] = '10-14'  THEN '0-19'
		WHEN [AGE_GROUP] = '0-14'  THEN '0-19'
        WHEN [AGE_GROUP] = '15-19'  THEN '0-19'        
		WHEN [AGE_GROUP] = '20-24' THEN '20-29'
		WHEN [AGE_GROUP] = '25-29' THEN '20-29'
		WHEN [AGE_GROUP] = '30-34' THEN '30-39'	
		WHEN [AGE_GROUP] = '35-39' THEN '30-39'	
		WHEN [AGE_GROUP] = '40-44' THEN '40-49'
		WHEN [AGE_GROUP] = '45-49' THEN '40-49'	
		WHEN [AGE_GROUP] = '50-54' THEN '50-59'
		WHEN [AGE_GROUP] = '55-59' THEN '50-59'
		WHEN [AGE_GROUP] = '60-64' THEN '60-69'
		WHEN [AGE_GROUP] = '65-69' THEN '60-69'
		WHEN [AGE_GROUP] = '70-74' THEN '70-79'
		WHEN [AGE_GROUP] = '75-79' THEN '70-79'
		WHEN [AGE_GROUP] = '80-84' THEN '80-89'
		WHEN [AGE_GROUP] = '85-89' THEN '80-89'
		WHEN [AGE_GROUP] = '90+'   THEN '90+'
		ELSE [AGE_GROUP]
		END AS [AGEGROUP]
      ,[HOSPITAL_ADMISSION_NOTIFICATION]
      ,[HOSPITAL_ADMISSION]
      ,[IC_ADMISSION_NOTIFICATION]
      ,[IC_ADMISSION] 
    FROM VWSINTER.RIVM_HOSPITAL_IC_ADMISSIONS_OVERTIME_BYAGEGROUP
    WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSINTER.RIVM_HOSPITAL_IC_ADMISSIONS_OVERTIME_BYAGEGROUP)
)
,TOTALS_CTE AS (
-- Totals per agegroup per date
SELECT 
	 [DATE_OF_REPORT]								
	,[DATE_OF_STATISTICS_WEEK_START]
	,[AGEGROUP]
    ,SUM([HOSPITAL_ADMISSION_NOTIFICATION])				AS [HOSPITAL_ADMISSION_NOTIFICATION]
    ,SUM([HOSPITAL_ADMISSION])							AS [HOSPITAL_ADMISSION]
    ,SUM([IC_ADMISSION_NOTIFICATION])					AS [IC_ADMISSION_NOTIFICATION]
    ,SUM([IC_ADMISSION])								AS [IC_ADMISSION]
	FROM BASE_CTE
	GROUP BY [DATE_OF_REPORT],[DATE_OF_STATISTICS_WEEK_START],[AGEGROUP]
UNION ALL 
-- Totals per date (values per agegroup summed)
SELECT 
	 [DATE_OF_REPORT]								
	,[DATE_OF_STATISTICS_WEEK_START]
	,'total'                                            AS [AGEGROUP]
    ,SUM([HOSPITAL_ADMISSION_NOTIFICATION])				AS [HOSPITAL_ADMISSION_NOTIFICATION]
    ,SUM([HOSPITAL_ADMISSION])							AS [HOSPITAL_ADMISSION]
    ,SUM([IC_ADMISSION_NOTIFICATION])					AS [IC_ADMISSION_NOTIFICATION]
    ,SUM([IC_ADMISSION])								AS [IC_ADMISSION]
	FROM BASE_CTE
	GROUP BY [DATE_OF_REPORT],[DATE_OF_STATISTICS_WEEK_START]
),
AGEGROUP_POPULATION AS (
SELECT 
	 AGEGROUP
	,DATUM_PEILING
	,POPULATIE
FROM VWSSTATIC.CBS_POPULATION_AGEGROUP_HOSPITAL_IC
WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSSTATIC.CBS_POPULATION_AGEGROUP_HOSPITAL_IC)
UNION ALL 
SELECT
	   'Total' AS [AGEGROUP]
      ,[DATUM_PEILING]
      ,[POPULATIE]
FROM [VWSSTATIC].[CBS_POPULATION_NL]
WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSSTATIC.CBS_POPULATION_NL)
)
,AGEGROUP_POPULATION_DATERANGE AS (
SELECT 
	 AGEGROUP
	,DATUM_PEILING AS DATE_FROM
    -- 2100-01-01 is used as convention for the most recent versions as no end date is known yet.
    ,ISNULL(LEAD(DATUM_PEILING,1) OVER (PARTITION BY AGEGROUP ORDER BY [DATUM_PEILING] ASC),CAST('2100-01-01' AS DATE)) AS DATE_TO
	,CAST(POPULATIE AS FLOAT) AS [POPULATION]
FROM AGEGROUP_POPULATION 
)
,
CALCULATED_CTE AS (
	SELECT
         [DATE_OF_REPORT]									AS [DATE_OF_REPORT]
        ,[DATE_OF_STATISTICS_WEEK_START]					AS [DATE_OF_STATISTICS_WEEK_START]
		,DATEADD(day,6,[DATE_OF_STATISTICS_WEEK_START])		AS [DATE_OF_STATISTICS_WEEK_END]		
        ,T0.AGEGROUP										AS [AGE_GROUP]
        ,[HOSPITAL_ADMISSION_NOTIFICATION]					AS [HOSPITAL_ADMISSION_NOTIFICATION]
        ,[HOSPITAL_ADMISSION]								AS [HOSPITAL_ADMISSION]
        ,[IC_ADMISSION_NOTIFICATION]						AS [IC_ADMISSION_NOTIFICATION]
        ,[IC_ADMISSION]										AS [IC_ADMISSION]
		,(1000000 / [POPULATION]) * [HOSPITAL_ADMISSION]	AS [HOSPITAL_ADMISSION_PER1M]
		,(1000000 / [POPULATION]) * [IC_ADMISSION]			AS [IC_ADMISSION_PER1M]

		--Additional data
		,ROW_NUMBER() OVER (PARTITION BY T0.AGEGROUP ORDER BY [DATE_OF_STATISTICS_WEEK_START] DESC ) AS LAST_WEEK_NR
		,T1.[POPULATION]
    FROM  TOTALS_CTE T0
    INNER JOIN AGEGROUP_POPULATION_DATERANGE T1 --only use known dateranges, neglect 'Unknown'
        ON  T0.AGEGROUP = T1.AGEGROUP
		-- Use population numbers at start of weekrange
        AND (T0.DATE_OF_STATISTICS_WEEK_START >= T1.DATE_FROM AND T0.DATE_OF_STATISTICS_WEEK_START < T1.DATE_TO)
)
--Determine ranges in which population numbers should be used
	INSERT INTO VWSDEST.RIVM_HOSPITAL_IC_ADMISSIONS_OVERTIME_BYAGEGROUP (
		[DATE_OF_REPORT]
		,[DATE_OF_STATISTICS_WEEK_START]
		,[DATE_OF_STATISTICS_WEEK_END]
		,[AGE_GROUP_GROUPED]
		,[HOSPITAL_ADMISSION_NOTIFICATION]
		,[HOSPITAL_ADMISSION]
		,[IC_ADMISSION_NOTIFICATION]
		,[IC_ADMISSION]
		,[HOSPITAL_ADMISSION_PER1M]
		,[IC_ADMISSION_PER1M]		
		)

SELECT 
         [DATE_OF_REPORT]
        ,[DATE_OF_STATISTICS_WEEK_START]
		,[DATE_OF_STATISTICS_WEEK_END]		
        ,[AGE_GROUP]
        ,[HOSPITAL_ADMISSION_NOTIFICATION]
        ,[HOSPITAL_ADMISSION]
        ,[IC_ADMISSION_NOTIFICATION]
        ,[IC_ADMISSION]
		,CAST([HOSPITAL_ADMISSION_PER1M]	as decimal(19,1))	AS [HOSPITAL_ADMISSION_PER1M]
		,CAST([IC_ADMISSION_PER1M]			as decimal(19,1))	AS [IC_ADMISSION_PER1M]
	FROM CALCULATED_CTE
	WHERE -- Remove records from last week if they contain NULLS
		LAST_WEEK_NR > 1 OR ([HOSPITAL_ADMISSION] IS NOT NULL AND IC_ADMISSION IS NOT NULL)
	ORDER BY DATE_OF_STATISTICS_WEEK_START, AGE_GROUP

END;
GO


## **TABLES**

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSDEST].[RIVM_HOSPITAL_IC_ADMISSIONS_OVERTIME_BYAGEGROUP]'))
BEGIN
    CREATE TABLE [VWSDEST].[RIVM_HOSPITAL_IC_ADMISSIONS_OVERTIME_BYAGEGROUP](
        [ID] [int] NOT NULL,
        [DATE_OF_REPORT] [datetime] NULL,
        [DATE_OF_STATISTICS_WEEK_START] [date] NULL,
        [DATE_OF_STATISTICS_WEEK_END] [date] NULL,
        [AGE_GROUP_GROUPED] [varchar](100) NULL,
        [HOSPITAL_ADMISSION_NOTIFICATION] [int] NULL,
        [HOSPITAL_ADMISSION] [int] NULL,
        [IC_ADMISSION_NOTIFICATION] [int] NULL,
        [IC_ADMISSION] [int] NULL,
        [HOSPITAL_ADMISSION_PER1M] [decimal](19, 1) NULL,
        [IC_ADMISSION_PER1M] [decimal](19, 1) NULL,
        [DATE_LAST_INSERTED] [datetime] NULL,
    PRIMARY KEY CLUSTERED 
    (
        [ID] ASC
    )WITH (STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]
    ) ON [PRIMARY]
    GO

    CREATE SEQUENCE [dbo].[SEQ_VWSDEST_RIVM_HOSPITAL_IC_ADMISSIONS_OVERTIME_BYAGEGROUP] 
    AS [bigint]
    START WITH 1
    INCREMENT BY 1
    MINVALUE -9223372036854775808
    MAXVALUE 9223372036854775807
    CACHE 
    GO

    ALTER TABLE [VWSDEST].[RIVM_HOSPITAL_IC_ADMISSIONS_OVERTIME_BYAGEGROUP] ADD  DEFAULT (NEXT VALUE FOR [DBO].[SEQ_VWSDEST_RIVM_HOSPITAL_IC_ADMISSIONS_OVERTIME_BYAGEGROUP]) FOR [ID]
    GO

    ALTER TABLE [VWSDEST].[RIVM_HOSPITAL_IC_ADMISSIONS_OVERTIME_BYAGEGROUP] ADD  DEFAULT (getdate()) FOR [DATE_LAST_INSERTED]
    GO


END


## **VIEWS**

In [ ]:
-- -- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- -- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

--1) CREATE VIEW(S).....
CREATE OR ALTER VIEW [VWSDEST].[V_HOSPITAL_ADMISSION_70_PLUS_OVER_TIME]
AS
	SELECT
			SUM([HOSPITAL_ADMISSION_NOTIFICATION] + [IC_ADMISSION_NOTIFICATION]) AS [ADMISSIONS_AGE_70_PLUS],
			[dbo].[CONVERT_DATETIME_TO_UNIX]([DATE_OF_STATISTICS_WEEK_START]) AS [DATE_START_UNIX],
			[dbo].[CONVERT_DATETIME_TO_UNIX]([DATE_OF_STATISTICS_WEEK_END]) AS [DATE_END_UNIX],
			[dbo].[CONVERT_DATETIME_TO_UNIX]([DATE_LAST_INSERTED]) AS [DATE_OF_INSERTION_UNIX]
	FROM [VWSDEST].[RIVM_HOSPITAL_IC_ADMISSIONS_OVERTIME_BYAGEGROUP]
	WHERE [AGE_GROUP_GROUPED] IN ('70-79', '80-89', '90+')
	AND [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSDEST].[RIVM_HOSPITAL_IC_ADMISSIONS_OVERTIME_BYAGEGROUP])
	GROUP BY [DATE_OF_STATISTICS_WEEK_START], [DATE_OF_STATISTICS_WEEK_END], [DATE_LAST_INSERTED]
GO




In [ ]:
-- -- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- -- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

--1) CREATE VIEW(S).....
CREATE OR ALTER VIEW [VWSDEST].[V_DIFFERENCES_HOSPITAL_ADMISSION_70_PLUS_OVER_TIME] AS 

WITH BASE_CTE AS 
(
    SELECT
		[DATE_START_UNIX] AS [DATE_UNIX],
		LAG([DATE_START_UNIX], 1) OVER (ORDER BY [DATE_START_UNIX]) AS [OLD_DATE_UNIX],
		[ADMISSIONS_AGE_70_PLUS] AS [NEW_VALUE],
		LAG([ADMISSIONS_AGE_70_PLUS], 1) OVER (ORDER BY [DATE_START_UNIX]) AS [OLD_VALUE],
		ROW_NUMBER() OVER (ORDER BY [DATE_START_UNIX] DESC) AS RANK_DATE
    FROM [VWSDEST].[V_HOSPITAL_ADMISSION_70_PLUS_OVER_TIME]
)
SELECT
    [DATE_UNIX] AS [NEW_DATE_UNIX],
    [OLD_DATE_UNIX] AS [OLD_DATE_UNIX],
    [OLD_VALUE],
    ROUND([NEW_VALUE] - [OLD_VALUE], 1) AS [DIFFERENCE]
FROM BASE_CTE
  WHERE RANK_DATE = 1 --Only the most recent record
GO




## **PROTOS**

Basic view

In [ ]:
-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_HOSPITAL_ADMISSION_70_PLUS_OVER_TIME',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'vulnerable_hospital_admissions',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = NULL,
        @grouped_last_update_name VARCHAR(50) = NULL,
        @proto_name VARCHAR(50) = 'NL',
        @columns VARCHAR(256) = 'ADMISSIONS_AGE_70_PLUS|DATE_START_UNIX|DATE_END_UNIX|DATE_OF_INSERTION_UNIX',
        @layout_type_id INT = 3, -- LASTVALUES
        @last_update_name VARCHAR(50) = 'DATE_START_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

DELETE FROM [DATATINO_PROTO_1].[CONFIGURATIONS]
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] = @proto_name
);

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO

difference

In [ ]:
-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_DIFFERENCES_HOSPITAL_ADMISSION_70_PLUS_OVER_TIME',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'difference/vulnerable_hospital_admissions',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = NULL,
        @grouped_last_update_name VARCHAR(50) = NULL,
        @proto_name VARCHAR(50) = 'NL',
        @columns VARCHAR(256) = 'NEW_DATE_UNIX|OLD_DATE_UNIX|OLD_VALUE|DIFFERENCE',
        @layout_type_id INT = 3, -- LASTVALUES
        @last_update_name VARCHAR(50) = 'NEW_DATE_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

DELETE FROM [DATATINO_PROTO_1].[CONFIGURATIONS]
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] = @proto_name
);

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO